## 使用nn.transformer和torchtext搭建语言模型
根据论文Attention is all your need，pytorch包含一个标准的transformer模块。比起RNN来说，transfromer在许多序列到序列的任务中具有更高的质量，同时具有更强的并行性。  

nn.Transformer模块完全依赖于一种注意力机制（nn.MulitiheadAttention）来控制输入和输出之间的全局依赖关系。  

Transformer模块是高度模块化的，因此单个组件(例如nn.TransformerEncoder)可以很容易地适配/组合。

## 定义模型
本次的语言模型任务是在一个单词序列出现后为给定单词（或单词序列）赋值一个概率。  

如框架图，一系列tokens首先传入embedding层，再进行positional encoding，以确定单词的顺序。  

nn.transformerEncoder由nn.TransformerEncoderLayer中的多层组成。  
在输入序列的同时，由于nn.transformerEncoder的自注意力层只允许出现在序列中较早的位置，因此需要一个正方形的attention mask。  

对于语言建模任务来说，任何在未来可能出现位置上的tokens都应该被mask。  
为了在输出单词上生成一个概率分布，nn.transformerEncoder模型的输出通过一个线性层后进行softmax操作。

In [1]:
import math
from typing import Tuple  # typing模块

import torch
# torch.nn和torch.nn.functional两个库都可以实现神经网络的各层运算。
# 其他包括卷积、池化、padding、激活(非线性层)、线性层、正则化层、其他损失函数Loss，两者都可以实现
# nn.functional毕竟只是nn的子库，nn的功能要多一些
from torch import nn, Tensor  
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):
    def __init__(self, ntoken:int, d_model:int, nhead:int, d_hid:int, 
                nlayers:int, dropout:float=0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()  # 参数初始化

    def init_weights(self) -> None:
        initrange = 0.1
        self.encode.weight.data.uniform_(-initrange,initrange)  # 权重随机生成一个实数范围在-0.1，0，1之间，_符号代表直接覆盖类型的函数
        self.decoder.bias.data.zero_()  # 偏置设为0
        self.decoder.weight.data.uniform_(-initrange,initrange)
    
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        参数：
            src:张量，[seq_len, batch_size]
            src_mask:张量，[seq_len, seq_len] 正方形mask
        返回：
            张量，形状，[seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)  # 位置编码
        output = self.transformer_encoder(src, src_mask)  # 个人感觉核心步骤都封装在在transformer_encoder和decoder里面
        output = self.decoder(output)

        return output
    
    def generate_square_subsequent_mask(sz:int) -> Tensor:
        # 生成一个上三角矩阵，值为-inf（无穷小）
        return torch.triu(torch.ones(sz, sz) * float('inf'), diagonal = 1)

## 位置编码模块注入一些关于序列中tokens的相对或绝对位置的信息。
## 位置编码与embedding具有相同的尺寸，以便两者可以求和。
## 这里，我们使用不同频率的正弦和余弦函数

In [12]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model:int, dropout:float=0.1, max_len:int=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)  # 生成纵向位置tensor
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))  # torch.arange(0, d_model, 2)得到一个step为2的等差数列，-math.log(10000.0)是固定值-9.
        pe = torch.zeros(max_len, 1, d_model)  # pe 3个维度的0张量
        pe[:, 0, 0::2] = torch.sin(position * div_term)  # 
        pe[:, 0, 1::2] = torch.cos(position * div_term)  # 
        self.register_buffer('pe', pe)  # 向模块添加缓冲区
    
    def forward(self, x:Tensor) -> Tensor:
        """
        参数：
            x:张量，[seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]  # x的0维度所有数据
        return self.dropout(x)
